In [11]:
%load_ext autoreload
%autoreload 2

import os
DATA_DIR = os.path.expanduser("~/repos/data/")
# DATA_DIR = "/data1/simulations/datasets/rotors/high_temp_ML_training_data/"

FANDE_DIR = os.path.expanduser("~/repos/")
# RESULTS_DIR = os.path.expanduser("~/repos/data/results")
# os.makedirs(RESULTS_DIR, exist_ok=True)

import os
import sys
import torch
sys.path.append(FANDE_DIR + "fande/")

In [12]:
%%time
from ase import io

traj_295 = io.read(DATA_DIR+"/results_triasine_ML_2000/struct_295_295K/md_trajectory.traj", index="0:10")
# traj_355 = io.read(DATA_DIR+"/results_triasine_ML_2000/struct_355_355K/md_trajectory.traj", index=":")
# traj_295_2000K = io.read(DATA_DIR+"/results_triasine_ML_2000/struct_295_2000K/md_trajectory.traj", index=":")
# traj_355_2000K = io.read(DATA_DIR+"/results_triasine_ML_2000/struct_355_2000K/md_trajectory.traj", index=":")
# traj_295_2000K_forced = io.read(DATA_DIR+"/results_triasine_ML_2000/struct_295_2000K_0075force/md_trajectory.traj", index=":")
# traj_355_2000K_forced = io.read(DATA_DIR+"/results_triasine_ML_2000/struct_355_2000K_0075force/md_trajectory.traj", index=":")

CPU times: user 68.5 ms, sys: 16.9 ms, total: 85.4 ms
Wall time: 98 ms


In [13]:
init_structure = traj_295[0]
input_xml_str = """
<simulation verbosity='high'>
  <output prefix='PREFIX'>
<!--    <properties stride='1' filename='out'>  [ step, potential{kilocal/mol} ] </properties>-->
    <!-- <trajectory stride='1' filename='pos'>  positions{angstrom}  </trajectory> -->
    <!-- <properties stride="10"> [ step, time, pot_component_raw(0), pot_component_raw(1), temperature, conserved ] </properties> -->
    <properties stride="1"> [ step, time, potential(0), temperature, conserved ] </properties>
    <trajectory stride="1" filename="pos" cell_units="angstrom"> positions{angstrom} </trajectory>
    <trajectory stride="1" filename="pos_centroid" cell_units="angstrom"> x_centroid{angstrom} </trajectory>
    <trajectory stride="1" filename="frc"> forces </trajectory>
    <checkpoint stride="1000" overwrite="False"/>
  </output>
  <total_steps> 1000 </total_steps>
  <ffsocket mode='inet' name='mycalc' pbc='True'>
    <address>localhost</address>
    <port>10200</port>
  </ffsocket>
  <system>
    <initialize nbeads='1'>
      <file mode='ase'>  /home/qklmn/data/starting_configuration/1.cif </file>
      <velocities mode="thermal" units="kelvin"> 20 </velocities>
    </initialize>
    <forces>
      <force forcefield='mycalc'> </force>
    </forces>
    <ensemble>
      <temperature units='kelvin'> 300.0 </temperature>
    </ensemble>
    <motion mode='dynamics'>
      <dynamics mode='nvt'>
        <timestep units='femtosecond'> 0.5 </timestep>
        <thermostat mode='langevin'>
          <tau units='femtosecond'>10</tau>
        </thermostat>
      </dynamics>
    </motion>
  </system>
</simulation>
"""

calc_dir = DATA_DIR + "/results/test_run/"
calculator = 'xtb'
num_instances = 1

In [20]:
from fande import ipi

In [21]:
ipi.launch(init_structure, calculator, num_instances, calc_dir, input_xml_str)

Working directory:  /home/mm/repos/data/results/test_run
Launched! Please manually check if i-pi is running.


sh: 1: source: not found


0

In [7]:
os.getcwd()

'/home/mm/repos/rotor-gp/code'

In [56]:
! source ~/repos/i-pi/env.sh; which i-pi

/home/mm/repos/i-pi/bin/i-pi


In [55]:
!which i-pi

In [60]:

import os

os.system("source ~/repos/i-pi/env.sh; which i-pi")


sh: 1: source: not found


256

In [98]:
import subprocess

proc = subprocess.run('. /home/mm/repos/i-pi/env.sh', shell=True)
# print(os.environ['FOO'])

/bin/sh: 1: /home/mm/repos/i-pi/env.sh: Bad substitution
/bin/sh: 3: export: Code/bin:/mnt/c/texlive/2022/bin/win32:/mnt/c/Users/tsits/AppData/Local/Programs/Microsoft: bad variable name


In [71]:
proc.stdout